# Old

### Utils

In [20]:
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pickle

def is_bottom_of_page(driver: webdriver.Chrome) -> bool:
    # Get the current scroll position and the total scrollable height
    current_scroll_position = driver.execute_script("return window.pageYOffset + window.innerHeight;")
    total_scroll_height = driver.execute_script("return document.body.scrollHeight;")
    return current_scroll_position >= total_scroll_height

def infinite_scroll(driver: webdriver.Chrome):
    while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            
            check_showmore = False
            try:
                element = driver.find_element(By.CLASS_NAME, value='''artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--full artdeco-button--secondary ember-view scaffold-finite-scroll__load-button''')
                check_showmore = True
            except NoSuchElementException as e:
                check_showmore = False
                pass

            if check_showmore:
                driver.execute_script('window.scrollTo(document.body.scrollHeight, 0);')
                sleep(2)
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

            if is_bottom_of_page(driver):
                break

def get_fullxpath(driver: webdriver.Chrome, element: WebElement) -> str:
    return driver.execute_script( 
     '''function getElementXPath(elt) {
            var path = "";
            for (; elt && elt.nodeType == 1; elt = elt.parentNode) {
                idx = getElementIdx(elt);
                xname = elt.tagName;
                if (idx > 1) xname += "[" + idx + "]";
                path = "/" + xname + path;
            }
            return path;
        }

        function getElementIdx(elt) {
            var count = 1;
            for (var sib = elt.previousSibling; sib; sib = sib.previousSibling) {
                if (sib.nodeType == 1 && sib.tagName == elt.tagName) count++;
            }
            return count;
        }

        return getElementXPath(arguments[0]);''',
        element
    ).lower()




def save_cookies(driver, cookies_file):
    with open(cookies_file, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

def load_cookies(driver, cookies_file):
    with open(cookies_file, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            driver.add_cookie(cookie)

def is_logged_in(driver):
    # Check if the user is logged in by looking for a specific element
    try:
        driver.find_element(By.ID, 'profile-nav-item')
        return True
    except:
        return False

## 

### LinkedInCrawler class

In [9]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
from bs4 import BeautifulSoup
from time import sleep

# from utils import infinite_scroll, get_fullxpath, is_bottom_of_page, get_urls


In [10]:
class LinkedInCrawler:
    driver: webdriver.Chrome
    connection_urls: list
    connection_info_cache: list

    def __init__(self):
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        self.connection_urls = []

    #region LOGIN LINKEDIN
    def login_another_account(self, username: str, password: str, cookies_file:str = 'linked_cookies.pkl'):
        sleep(1.5)
        url = 'https://www.linkedin.com/checkpoint/lg/sign-in-another-account'
        self.driver.get(url)
        print('- Finish initializing a self.driver')
        sleep(1.5)

        print('- Finish importing the login credentials')
        sleep(1.5)

        email_field = self.driver.find_element(by=By.ID, value="username")
        email_field.send_keys(username)
        print('- Finish keying in email')
        sleep(1.5)

        password_field = self.driver.find_element(by=By.NAME, value="session_password")
        password_field.send_keys(password)
        password_field.send_keys
        print('- Finish keying in password')
        sleep(1.5)

        signin_field = self.driver.find_element(by=By.XPATH, value='//*[@id="organic-div"]/form/div[3]/button')
        signin_field.click()
        print('- Finish Task: Login to Linkedin')
        sleep(2)
        save_cookies(self.driver, cookies_file)
        sleep(1)
    #endregion

    def login_current_account(self, username, password):
        sleep(1.5)
        url = 'https://www.linkedin.com/login'
        self.driver.get(url)

        


    #region GET CONNECTIONS URLs
    def get_connection_urls(self):
        sleep(1)
        # my_network_btn = self.driver.find_element(By.XPATH, value='//*[@id="global-nav"]/div/nav/ul/li[2]/a')
        # wait = WebDriverWait(self.driver, 10)
        # wait.until(EC.element_to_be_clickable(my_network_btn))
        # my_network_btn.click()

        # sleep(1)
        # connections_btn = self.driver.find_element(By.XPATH, value='//*[@id="root"]/div[3]/div[2]/div/div/div/section/div/div/section[1]/div/nav/ul/li[1]/a')
        # wait = WebDriverWait(self.driver, 10)
        # wait.until(EC.element_to_be_clickable(connections_btn))
        # connections_btn.click()
        self.driver.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')
        #scroll loop
        while True:
                sleep(1)
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                sleep(1)
                
                check_showmore = False
                try:
                    element = self.driver.find_element(By.CLASS_NAME, value='''artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--full artdeco-button--secondary ember-view scaffold-finite-scroll__load-button''')
                    check_showmore = True

                except NoSuchElementException as e:
                    check_showmore = False
                    pass

                if check_showmore:
                    self.driver.execute_script('window.scrollTo(document.body.scrollHeight, 0);')
                    self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

                if is_bottom_of_page(self.driver):
                    print("Reached the bottom of the page.")
                    break


        urls = get_urls(self.driver)
        self.connection_urls = urls
    #endregion


    #region GET CONTACT INFO
    def get_contact_info(self):
        try:
            sleep(1)
            self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')     
            sleep(0.5)   
            self.driver.execute_script('window.scrollTo(document.body.scrollHeight,0);')
            sleep(0.5)
            a_contact_info = self.driver.find_element(By.ID, value='''top-card-text-details-contact-info''')
            wait = WebDriverWait(self.driver, 10)
            wait.until(EC.element_to_be_clickable(a_contact_info))
            a_contact_info.click()

            sleep(1.5)
            div_profile_section = self.driver.find_element(By.XPATH, value='''/html/body/div[4]/div/div''')
            div_profile_section_source = BeautifulSoup(div_profile_section.get_attribute('outerHTML'),'html.parser')
            h1_tags = div_profile_section_source.find_all('h1')
            h1_text = [tag.text.strip() for tag in h1_tags]
            h3_tags = div_profile_section_source.find_all('h3')
            h3_text = [tag.text.strip() for tag in h3_tags]
            a_tags = div_profile_section_source.find_all('a')
            a_text = [tag.text.strip() for tag in a_tags]
            span_tags = div_profile_section_source.find_all('span')
            span_text = [tag.text.strip() for tag in span_tags]
            
            exit_btn = self.driver.find_element(By.XPATH, value='''/html/body/div[4]/div/div/button''')
            exit_btn.click()

            return {
                'h1_tag_text': h1_text,
                'h3_tag_text': h3_text,
                'a_tag_text': a_text,
                'span_tang_text': span_text
            }
        except NoSuchElementException as e:
            print(e)    
    #endregion


    #region NOT USED
    # #region GET AVAILABLE SECTIONS ON A PROFILE
    # def get_sections(self) -> dict:
    #     sleep(1)
    #     section_head = {}
    #     main_element = self.driver.find_element(By.CLASS_NAME, value='scaffold-layout__main')
    #     main_element_source = BeautifulSoup(main_element.get_attribute('outerHTML'),'html.parser')
    #     sections = main_element_source.find_all('section', class_='artdeco-card pv-profile-card break-words mt2')
    #     for i in range(0,len(sections)):
    #         if len([tag.text.strip() for tag in sections[i].select('span.visually-hidden')]) > 0:
    #             head = ([tag.text.strip() for tag in sections[i].select('span.visually-hidden')][0])
    #             section_head[head] = i+2
    #     return section_head
    # #endregion


    # #region GET SECTION's INFO
    # def get_section_info(self, section_number: int, full_xpath: str ):
    #     sleep(2)
    #     wait = WebDriverWait(self.driver,10)
    #     info_list = []
    #     show_more_check = False
    #     try:
    #         show_more = self.driver.find_element(By.XPATH, value= full_xpath + f'''/section[{section_number}]/div[3]/div/div/div/a''')
    #         show_more_check = True
    #     except NoSuchElementException:
    #         show_more_check = False

    #     if show_more_check:   #if yes: click show more
    #         click_count = 0
    #         while(True):
    #             try:
    #                 self.driver.execute_script(f"window.scrollTo(0, {show_more.size['height']});")
    #                 wait.until(EC.element_to_be_clickable(show_more))
    #                 show_more.click()
    #                 sleep(2)
    #             except Exception as e:
    #                 click_count += 1
    #                 print(e)
                
    #             if click_count > 0:
    #                 continue
    #             else:
    #                 break

    #         infinite_scroll(self.driver)
    #         list_container = self.driver.find_element(By.CLASS_NAME, value='pvs-list__container')
    #         list_container_source = BeautifulSoup(list_container.get_attribute('outerHTML'),'html.parser')
    #         li_elements = list_container_source.find_all('div', class_='display-flex flex-column align-self-center flex-grow-1')

    #         for li in li_elements:
    #             span_tags = li.find_all('span', class_='visually-hidden')
    #             span_text = [tag.text.strip() for tag in span_tags]
    #             info_list.append(span_text) 
                    
    #         self.driver.back()
            
    #     else:
    #         fail_count = 0
    #         while (fail_count<5):
    #             try:
    #                 ul_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul')
    #                 break
    #             except NoSuchElementException as e:
    #                 fail_count += 1

    #         if(fail_count==5):
    #             ul_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul')
                

    #         ul_element_source = BeautifulSoup(ul_element.get_attribute('outerHTML'),'html.parser')
    #         number_of_li = len(ul_element_source.find_all('li',class_='artdeco-list__item'))

    #         if(number_of_li<2):
    #             # li_element = driver.find_element(By.XPATH, value='/html/body/div[6]/div[3]/div/div/div[2]/div/div/main/section[3]/div[3]/ul/li/div/div[2]')
    #             li_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul/li/div/div[2]')
    #             li_element_source = BeautifulSoup(li_element.get_attribute('outerHTML'),'html.parser')
    #             span_tags = li_element_source.find_all('span', class_='visually-hidden')
    #             span_text = [tag.text.strip() for tag in span_tags]
    #             info_list.append(span_text)
    #         else:
    #             for j in range(number_of_li):
    #                 li_number = j+1
    #                 # li_element = driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul/li[{li_number}]/div/div[2]')
    #                 li_element = self.driver.find_element(By.XPATH, value=f'//*[@id="profile-content"]/div/div[2]/div/div/main/section[{section_number}]/div[3]/ul/li[{li_number}]/div/div[2]')
    #                 li_element_source = BeautifulSoup(li_element.get_attribute('outerHTML'),'html.parser')
    #                 span_tags = li_element_source.find_all('span', class_='visually-hidden')
    #                 span_text = [tag.text.strip() for tag in span_tags]
    #                 info_list.append(span_text)    

    #     return info_list
    # #endregion


    # #region GET EXPERIENCE
    # def get_experience(self,sections: dict, full_xpath: str) -> list:
    #     if 'Experience' in sections:
    #         section_number = sections['Experience']
    #         section_info = self.get_section_info(section_number, full_xpath)
    #         return section_info

    #     return None
    # #endregion


    # #region GET SKILLS
    # def get_skills(self, sections: dict, full_xpath: str) -> list:
    #     if 'Skills' in sections:
    #         section_number = sections['Skills']
    #         section_info = self.get_section_info(section_number, full_xpath)
    #         return section_info

    #     return None
    # #endregion


    # #region GET EDUCATION
    # def get_education(self, sections: dict, full_xpath: str) -> list:
    #     if 'Education' in sections:
    #         section_number = sections['Education']
    #         section_info = self.get_section_info(section_number, full_xpath)
    #         return section_info

    #     return None
    # #endregion


    # #region GET CERTIFICATION
    # def get_cert(self, sections: dict, full_xpath: str) -> list:
    #     if 'Licenses & certifications' in sections:
    #         section_number = sections['Licenses & certifications']
    #         section_info = self.get_section_info(section_number, full_xpath)
    #         return section_info

    #     return None
    # #endregion


    # #region GET PROFILE INFO
    # def get_profile_info(self, profile_url) -> dict:
    #     self.driver.get(profile_url)
    #     main_element = self.driver.find_element(By.CLASS_NAME, value='scaffold-layout__main')
    #     full_xpath = get_fullxpath(self.driver,main_element)
    #     sections = self.get_sections()
    #     profile_info = {
    #         'Contact Info': self.get_contact_info(),
    #         'Experience': self.get_experience(sections, full_xpath),
    #         'Education': self.get_education(sections, full_xpath),
    #         'Skills': self.get_skills(sections, full_xpath),
    #         'Licenses & certifications': self.get_cert(sections, full_xpath)
    #     }

    #     return profile_info
    # #endregion


    # #region GET MUILIPLE PROFILES
    # def get_profile_info_multi(self):
    #     connection_info_list = []
    #     i = 1
    #     for profile_url in self.connection_urls:
    #         profile_info = self.get_profile_info(profile_url)
    #         connection_info_list.append(profile_info)
    #         print(f"GET profile {i}")
    #         i += 1

    #     self.connection_info_cache = connection_info_list
    #     return connection_info_list
    # #endregion
    #endregion


    #region SEND MESSAGE
        


### TEST

In [11]:
# URL = 'https://www.linkedin.com/in/selenado/'
# URL = 'https://www.linkedin.com/in/trieuchinh/'
# URL = 'https://www.linkedin.com/in/tr%E1%BA%A7n-ho%C3%A0ng-qu%C3%A2n-37817ab6/'
# URL = 'https://www.linkedin.com/in/truong-nguyen-nhat-a5b77878/'
URL = 'https://www.linkedin.com/in/le-ha-b7a86997/'
# password = "prodn123"
# username = "skytdv123@gmail.com"
password = '3Contrau!'
username = 'quyendo00.ai@gmail.com'

#### Test get section info method

In [47]:
crawler = LinkedInCrawler()
crawler.login_linkedin(username,password)
crawler.driver.get(URL)
sections = crawler.get_sections()
main_element = crawler.driver.find_element(By.CLASS_NAME, value='scaffold-layout__main')
full_xpath = get_fullxpath(crawler.driver, main_element)
section_info = crawler.get_cert(sections, full_xpath)

- Finish initializing a self.driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task: Login to Linkedin
Reached the bottom of the page.


#### Test get 1 profile info

In [62]:
profile_info = crawler.get_profile_info(URL)

#### Test get muiltiple profiles

In [12]:
crawler = LinkedInCrawler()
crawler.login_linkedin(username,password)
# crawler.get_connection_urls()

- Finish initializing a self.driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task: Login to Linkedin


In [81]:

{
    'Name': contact_info['h1_tag_text'][0],
    'LinkedIn': contact_info['a_tag_text'][0],
    'Email': contact_info['a_tag_text'][1]
}

{'Name': 'Trần Hoàng Quân',
 'LinkedIn': 'linkedin.com/in/trần-hoàng-quân-37817ab6',
 'Email': 'hoangquan1991@gmail.com'}

In [82]:
contact_info_list = []
for profile in connection_info_list:
    contact_info = profile['Contact Info']
    contact_info_list.append({
    'Name': contact_info['h1_tag_text'][0],
    'LinkedIn': contact_info['a_tag_text'][0],
    'Email': contact_info['a_tag_text'][1]
})


#### Test send message

In [15]:
crawler.driver.get('https://www.linkedin.com/in/selenado/')
message = '''asdasdjnasdjasdnasdjasdiasjdjmkl
asdasdasdasdsadasasdasdmask
asdaskdomaskdasdoklmaskdmasd
asdmasdklasmdklmasklsmaklmas
'''
try:
    sleep(3)
    full_xpath = get_fullxpath()
    message_btn = crawler.driver.find_element(By.XPATH, value='/html/body/div[6]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[1]/button')
    wait = WebDriverWait(crawler.driver, 10)
    wait.until(EC.element_to_be_clickable(message_btn))
    message_btn.click()
    sleep(2)

    message_form = crawler.driver.find_element(By.CLASS_NAME, value='msg-form__contenteditable t-14 t-black--light t-normal flex-grow-1 full-height notranslate')
    message_form.send_keys(message)

except Exception as e:
    print(e)

# New

## LinkedIn Agent

### Utils

In [1]:
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import pickle
import os

def locate_element(driver: webdriver.Chrome, by: str, value: str) -> WebElement:
    sleep(0.5)
    try:
        element = driver.find_element(by,value)
        return element
    except NoSuchElementException:
        return None
    
def save_cookies(driver: webdriver.Chrome, cookies_file: str):
    with open(cookies_file, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

def load_cookies(driver: webdriver.Chrome, cookies_file: str):
    with open(cookies_file, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            driver.add_cookie(cookie)

def get_fullxpath(driver: webdriver.Chrome, element: WebElement) -> str:
    return driver.execute_script( 
     '''function getElementXPath(elt) {
            var path = "";
            for (; elt && elt.nodeType == 1; elt = elt.parentNode) {
                idx = getElementIdx(elt);
                xname = elt.tagName;
                if (idx > 1) xname += "[" + idx + "]";
                path = "/" + xname + path;
            }
            return path;
        }

        function getElementIdx(elt) {
            var count = 1;
            for (var sib = elt.previousSibling; sib; sib = sib.previousSibling) {
                if (sib.nodeType == 1 && sib.tagName == elt.tagName) count++;
            }
            return count;
        }

        return getElementXPath(arguments[0]);''',
        element
    ).lower()

def is_bottom_of_page(driver: webdriver.Chrome) -> bool:
    # Get the current scroll position and the total scrollable height
    current_scroll_position = driver.execute_script("return window.pageYOffset + window.innerHeight;")
    total_scroll_height = driver.execute_script("return document.body.scrollHeight;")
    return current_scroll_position >= total_scroll_height

### LinkedInAgent

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
from bs4 import BeautifulSoup
from time import sleep
import os

# from utils import infinite_scroll, get_fullxpath, is_bottom_of_page, get_urls

In [3]:
USERNAME = "quyendo00.ai@gmail.com"
PASSWORD = "3Contrau!"

class LinkedInAgent:
    driver: webdriver.Chrome
    connection_urls: list = None
    linked_cookies_path: str = 'linkedin_cookies.pkl'

    def __init__(self):
        self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

    #region IS LOGIN ?
    def is_logged_in(self) -> bool:
        # Check if the user is logged in by looking for a specific element
        div_body = locate_element(self.driver, By.CLASS_NAME, value='application-outlet')
        if div_body:
            return True
        else:
            return False

    #endregion


    #region LOGIN ANOTHER ACC
    def login_another_account(self, username: str = USERNAME, password: str = PASSWORD):
        wait = WebDriverWait(self.driver,15)
        url = 'https://www.linkedin.com/checkpoint/lg/sign-in-another-account'
        self.driver.get(url)
        sleep(3)

        email_field = self.driver.find_element(by=By.ID, value="username")
        email_field.send_keys(username)
        sleep(1)

        password_field = self.driver.find_element(by=By.NAME, value="session_password")
        password_field.send_keys(password)
        password_field.send_keys
        sleep(1)

        signin_field = self.driver.find_element(by=By.XPATH, value='//*[@id="organic-div"]/form/div[3]/button')
        signin_field.click()
        sleep(2)
        save_cookies(self.driver, self.linked_cookies_path)
        
    #endregion


    #region LOGIN CURRENT ACC
    def login_current_account(self, username: str=USERNAME, password: str=PASSWORD):
        url = 'https://www.linkedin.com/login'
        self.driver.get(url)

        if os.path.exists(self.linked_cookies_path):
            # Load cookies
            load_cookies(self.driver, self.linked_cookies_path)
            self.driver.refresh()                

        if not (self.is_logged_in()):
            self.login_another_account(username,password)
        else:
            sleep(2)
    #endregion        

    #region GET URLS
    def get_urls(self):
        page_source = BeautifulSoup(self.driver.page_source)
        profiles = page_source.find_all('a', class_ = 'ember-view mn-connection-card__link')
        all_profile_URL = []
        for profile in profiles:
            profile_ID = profile.get('href')
            profile_URL = "https://www.linkedin.com" + profile_ID
            if profile_URL not in all_profile_URL:
                all_profile_URL.append(profile_URL)
        return all_profile_URL       
    #endregion
     
    #region GET CONNECTIONS URLs
    def get_connection_urls(self) -> list[str]:
        wait = WebDriverWait(self.driver, 15)
        sleep(1)
        self.driver.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')
        wait.until(EC.presence_of_element_located((By.ID,'mynetwork')))

        #scroll loop
        while True:
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                sleep(2)
                showmore_btn = locate_element(self.driver, By.CLASS_NAME, value='artdeco-button artdeco-button--muted artdeco-button--1 artdeco-button--full artdeco-button--secondary ember-view scaffold-finite-scroll__load-button')

                if showmore_btn:
                    self.driver.execute_script('window.scrollTo(document.body.scrollHeight, document.body.scrollHeight-500);')
                    self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

                if is_bottom_of_page(self.driver):
                    print("Reached the bottom of the page.")
                    break
    
        urls = self.get_urls()
        self.connection_urls = urls
        return urls
    #endregion

    def navigate_to_profile(self, profile_url: str):
        wait = WebDriverWait(self.driver, 15)
        self.driver.get(profile_url)

        if not (self.is_logged_in()):
            self.login_another_account()
            self.driver.get(profile_url)

        wait.until(EC.presence_of_element_located((By.ID,'profile-content')))

    def message_action(self, message: str):
        wait = WebDriverWait(self.driver, 15)
        div_body = locate_element(self.driver, By.CLASS_NAME, 'body')
        body_xpath = get_fullxpath(self.driver, div_body)
        print(body_xpath)

        message_btn = locate_element(self.driver, By.XPATH, body_xpath+'/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[1]/button')
        wait.until(EC.element_to_be_clickable(message_btn))
        message_btn.click()


        overlay_container = locate_element(self.driver, By.CLASS_NAME, 'application-outlet__overlay-container')
        overlay_container_xpath = get_fullxpath(self.driver, overlay_container)

        message_form_old = locate_element(self.driver, By.XPATH, overlay_container_xpath+"/aside[1]/div[2]/div[1]/div[2]/div/form/div[3]/div[1]/div/div[1]")
        if (message_form_old):
            message_form_old.send_keys(message)

        message_form_new = locate_element(self.driver, By.XPATH, overlay_container_xpath+"/aside[1]/div[2]/div[1]/div[2]/div/form/div[2]/div[1]/div/div[1]")
        if (message_form_new):
            message_form_new.send_keys(message)

        send_btn = self.driver.find_element(By.XPATH, value=overlay_container_xpath+'/aside[1]/div[2]/div[1]/div[2]/div/form/footer/div[2]/div[1]/button')
        wait.until(EC.element_to_be_clickable(send_btn))
        send_btn.click()

        close_btn = locate_element(self.driver, By.XPATH, overlay_container_xpath+'/aside[1]/div[2]/div[1]/header/div[4]/button[3]')
        wait.until(EC.element_to_be_clickable(close_btn))
        close_btn.click()    

    def send_message(self, message: str, profile_url: str):
        self.navigate_to_profile(profile_url)
        sleep(5)

        if (self.is_logged_in()):
            self.message_action(message)

        else:
            self.login_current_account(USERNAME,PASSWORD)
            self.navigate_to_profile(profile_url)
            self.message_action(message)

        



In [4]:
password = '3Contrau!'
username = 'quyendo00.ai@gmail.com'

message = '''sadasdasdasdasdas
asd
asdas
das
d

dasasasdasdasdasd

'''
profile_url = 'https://www.linkedin.com/in/selenado/'
agent = LinkedInAgent()
agent.login_current_account(username)
agent.send_message(message, profile_url)

/html/body/div[6]/div[3]/div/div


# Agent

#### utils.py

In [25]:
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from time import sleep
import pickle

def locate_element(driver: webdriver.Chrome, by: str, value: str) -> WebElement:
    sleep(0.5)
    try:
        element = driver.find_element(by,value)
        return element
    except NoSuchElementException:
        return None
    
def save_cookies(driver: webdriver.Chrome, cookies_file: str):
    with open(cookies_file, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)

def load_cookies(driver: webdriver.Chrome, cookies_file: str):
    with open(cookies_file, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            driver.add_cookie(cookie)

def get_fullxpath(driver: webdriver.Chrome, element: WebElement) -> str:
    return driver.execute_script( 
     '''function getElementXPath(elt) {
            var path = "";
            for (; elt && elt.nodeType == 1; elt = elt.parentNode) {
                idx = getElementIdx(elt);
                xname = elt.tagName;
                if (idx > 1) xname += "[" + idx + "]";
                path = "/" + xname + path;
            }
            return path;
        }

        function getElementIdx(elt) {
            var count = 1;
            for (var sib = elt.previousSibling; sib; sib = sib.previousSibling) {
                if (sib.nodeType == 1 && sib.tagName == elt.tagName) count++;
            }
            return count;
        }

        return getElementXPath(arguments[0]);''',
        element
    ).lower()

def is_bottom_of_page(driver: webdriver.Chrome) -> bool:
    # Get the current scroll position and the total scrollable height
    current_scroll_position = driver.execute_script("return window.pageYOffset + window.innerHeight;")
    total_scroll_height = driver.execute_script("return document.body.scrollHeight;")
    return current_scroll_position >= total_scroll_height

#### webdriver.py

In [26]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from time import sleep
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from core.config import settings

USERNAME = settings.LINKEDIN_USERNAME
PASSWORD = settings.LINKEDIN_PASSWORD
COOKIE_PATH = settings.COOKIE_PATH
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration (often recommended for headless)
chrome_options.add_argument("--window-size=1920x1080")  # Set window size to ensure all content is loaded

def is_logged_in(driver) -> bool:
    # Check if the user is logged in by looking for a specific element
    div_body = locate_element(driver=driver, by=By.CLASS_NAME, value='application-outlet')
    if div_body:
        return True
    else:
        return False
    
#region LOGIN ANOTHER ACC
def login_another_account(driver: webdriver.Chrome, username: str = USERNAME, password: str = PASSWORD, cookie_path: str = COOKIE_PATH):
    wait = WebDriverWait(driver,15)
    url = 'https://www.linkedin.com/checkpoint/lg/sign-in-another-account'
    driver.get(url)
    sleep(3)

    email_field = driver.find_element(by=By.ID, value="username")
    email_field.send_keys(username)
    sleep(1)

    password_field = driver.find_element(by=By.NAME, value="session_password")
    password_field.send_keys(password)
    password_field.send_keys
    sleep(1)

    signin_field = driver.find_element(by=By.XPATH, value='//*[@id="organic-div"]/form/div[3]/button')
    signin_field.click()
    sleep(2)
    save_cookies(driver, cookie_path)
#endregion

#region LOGIN CURRENT ACC
def login_current_account(driver: webdriver.Chrome, username: str=USERNAME, password: str=PASSWORD, cookie_path: str = COOKIE_PATH):
    url = 'https://www.linkedin.com/login'
    driver.get(url)

    if os.path.exists(cookie_path):
        # Load cookies
        load_cookies(driver=driver, cookies_file=cookie_path)
        driver.refresh()                

    if not (is_logged_in(driver=driver)):
        login_another_account(driver=driver)
    else:
        sleep(2)
#endregion     

def get_driver() -> webdriver.Chrome: # type: ignore
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    # driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

    login_current_account(driver=driver)

    return driver

#### init driver

In [57]:
chrome_driver = get_driver()


#### send_message.py

In [58]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep

from llama_index.core.tools import FunctionTool
from llama_index.core.tools import ToolMetadata

# chrome_driver = get_driver()

def navigate_to_profile(profile_url: str, driver: webdriver.Chrome = chrome_driver):
    wait = WebDriverWait(driver, 15)
    driver.get(profile_url)
    wait.until(EC.presence_of_element_located((By.ID,'profile-content')))

def message_action(message: str, driver: webdriver.Chrome = chrome_driver):
    wait = WebDriverWait(driver, 15)
    div_body = locate_element(driver, By.CLASS_NAME, 'body')
    body_xpath = get_fullxpath(driver, div_body)

    message_btn = locate_element(driver, By.XPATH, body_xpath+'/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[1]/button')
    wait.until(EC.element_to_be_clickable(message_btn))
    message_btn.click()


    overlay_container = locate_element(driver, By.CLASS_NAME, 'application-outlet__overlay-container')
    overlay_container_xpath = get_fullxpath(driver, overlay_container)

    message_form_old = locate_element(driver, By.XPATH, overlay_container_xpath+"/aside[1]/div[2]/div[1]/div[2]/div/form/div[3]/div[1]/div/div[1]")
    if (message_form_old):
        message_form_old.send_keys(message)

    message_form_new = locate_element(driver, By.XPATH, overlay_container_xpath+"/aside[1]/div[2]/div[1]/div[2]/div/form/div[2]/div[1]/div/div[1]")
    if (message_form_new):
        message_form_new.send_keys(message)

    send_btn = driver.find_element(By.XPATH, value=overlay_container_xpath+'/aside[1]/div[2]/div[1]/div[2]/div/form/footer/div[2]/div[1]/button')
    wait.until(EC.element_to_be_clickable(send_btn))
    send_btn.click()

    sleep(2)
    close_btn = locate_element(driver, By.XPATH, overlay_container_xpath+'/aside[1]/div[2]/div[1]/header/div[4]/button[3]')
    wait.until(EC.element_to_be_clickable(close_btn))
    close_btn.click()


def send_message( message: str, profile_url: str):
    navigate_to_profile(profile_url)
    sleep(5)
    message_action(message)
    # chrome_driver.quit()


def send_message_multiple(msg_url: dict):
    for message, url in msg_url.items():
        send_message(message,url)
    # chrome_driver.quit()

    
def send_message_tool():
    tool = FunctionTool.from_defaults(
        fn = send_message_multiple,
        tool_metadata=ToolMetadata(
            name = "send_linkedin_msg",
            description=(
                f"Used to send a message. Input is generated by an llm"
            )
        )
    )

    return tool


### Test

In [54]:
navigate_to_profile(driver=chrome_driver, profile_url='https://www.linkedin.com/in/selenado/')
message = 'Alo 1234'

In [43]:
wait = WebDriverWait(chrome_driver, 15)
overlay_container = locate_element(chrome_driver, By.CLASS_NAME, 'application-outlet__overlay-container')
overlay_container_xpath = get_fullxpath(chrome_driver, overlay_container)
print(overlay_container_xpath)
close_btn = locate_element(chrome_driver, By.XPATH, overlay_container_xpath+'/aside[1]/div[2]/div[1]/header/div[4]/button[3]')
wait.until(EC.element_to_be_clickable(close_btn))
close_btn.click()

/html/body/div[6]/div[4]


In [60]:
send_message(message=message, profile_url=profile_url)

TypeError: selenium.webdriver.remote.webdriver.WebDriver.find_element() argument after * must be an iterable, not NoneType

In [ ]:
def write_email(input: str) -> str:
    return input + "success"

write_email_tool = FunctionTool.from_defaults(
    fn=write_email,
    tool_metadata=ToolMetadata(
        name="write_email",
        description=(
            f"Used to write an email. Input is generated email from llm"
        ),
    ),
)

agent = OpenAIAgent.from_tools(
    tools=[write_email_tool],
    system_prompt="write marketing email",
    chat_history=history,
    llm=llm
)